# Conformal Prediction

Core reference: Shafer and Vovk, 2008. A Tutorial on Conformal Prediction, _Journal of Machine Learning Research, 9_, 371-421.

Basic problem set up: we observe items sequentially. Each item has a label, $y$, which could be numerical or categorical.
There is some black-box model that makes a prediction $\hat{y}$ of the value of $y$ for the next item, 
generally using side information $x$ (e.g., covariates).
The predictor can "learn" from previous cases: it does not have to stay the same over time.
After each prediction $\hat{y}$ is made, the true label $y$ is revealed.
Thus,  $(x_1, y_1), \ldots, (x_{n−1}, y_{n−1})$ and $x_n$ are available to predict $y_n$.

The problem is: given an acceptable error probability $\epsilon$, construct a "prediction region" $\Gamma^\epsilon$ of labels that contains the label of the next item with probability at least $1-\epsilon$.

The basic assumption of conformal prediction is that the cases are _exchangeable_, which is true if they are IID. 
(There has been work on weakening that assumption: see, e.g., Foygel, R.B., E.J. Candes, A. Ramdas, and R.J. Tibshirani, 2022.  Conformal prediction beyond exchangeability, https://arxiv.org/abs/2202.13415.)

Conformal prediction works for any prediction method, though: regression, Bayesian, support-vector machines, neural networks, etc. 

Recall [the definition of a _multiset_ or _bag_](./math-foundations.ipynb#Multisets): like a set, it is a collection of things, but unlike a set, it can contain multiple copies of the same thing. It amounts to an unordered list.